In [3]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
#train file
path = "dataset/train.csv"
df_train = pd.read_csv(path)
df_train = df_train.dropna()
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,<p><strong>Aluminum Foil Stickers-good kitchen...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787


In [5]:
#test file
path = "dataset/test.csv"
df_test = pd.read_csv(path)
df_test.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [6]:
punctuation_signs = list("?:!.,;")
nltk.download('punkt')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arpit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df_train['Title'] = df_train['TITLE'].str.replace("\r", " ")
df_train['Title'] = df_train['Title'].str.replace("\n", " ")
df_train['Title'] = df_train['Title'].str.replace("    ", " ")
df_train['Title'] = df_train['Title'].str.replace('"', '')
df_train['Title'] = df_train['Title'].str.lower()
for punct_sign in punctuation_signs:
  df_train['Title'] = df_train['Title'].str.replace(punct_sign, '')
df_train['Title'] = df_train['Title'].str.replace("'s", "")

C:\Users\arpit\AppData\Local\Temp\ipykernel_5700\2045726992.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train['Title'] = df_train['Title'].str.replace(punct_sign, '')


In [8]:
final_cols = ["Title", "PRODUCT_LENGTH"]
df_train = df_train[final_cols]
df_train = df_train.iloc[:35000, :]

In [9]:
df_test['Title'] = df_test['TITLE'].str.replace("\r", " ")
df_test['Title'] = df_test['Title'].str.replace("\n", " ")
df_test['Title'] = df_test['Title'].str.replace("    ", " ")
df_test['Title'] = df_test['Title'].str.replace('"', '')
df_test['Title'] = df_test['Title'].str.lower()
for punct_sign in punctuation_signs:
  df_test['Title'] = df_test['Title'].str.replace(punct_sign, '')
df_test['Title'] = df_test['Title'].str.replace("'s", "")

C:\Users\arpit\AppData\Local\Temp\ipykernel_5700\1691348140.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_test['Title'] = df_test['Title'].str.replace(punct_sign, '')


In [10]:
final_cols = ["Title", "PRODUCT_ID"]
df_test = df_test[final_cols]

In [11]:
df_train.head()

,Title,PRODUCT_LENGTH
2,priknik horn red electric air horn compressor ...,748.031495
3,alishah women cotton ankle length leggings com...,787.401574
5,hins metal bucket shape plant pot for indoor &...,950.000000
7,delavala self adhesive kitchen backsplash wall...,984.251967
9,hexwell essential oil for home fragrance oil a...,393.700787


In [12]:
df_test.head()

,Title,PRODUCT_ID
0,manuel d'héliogravure et de photogravure en re...,604373
1,dcgaring microfiber throw blanket warm fuzzy p...,1729783
2,i-match auto parts front license plate bracket...,1871949
3,pinmart gold plated excellence in service 1 ye...,1107571
4,visual mathematics illustrated by the ti-92 an...,624253


In [13]:
df_train.isna().sum()

Title             0
PRODUCT_LENGTH    0
dtype: int64

In [14]:
df_test["Title"].fillna("No Data", inplace = True)

In [15]:
df_test.isna().sum()

Title         0
PRODUCT_ID    0
dtype: int64

In [16]:
X_train, X_test, y_train, y_test = df_train["Title"], df_train["Title"], df_train["PRODUCT_LENGTH"]

In [17]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
print('1')
X_train_vectors_tfidf = tfidf.fit_transform(X_train)
print(X_train_vectors_tfidf.shape)
print('1')
X_test_vectors_tfidf = tfidf.transform(X_test)
print(X_test_vectors_tfidf.shape)

1
(35000, 19349)
1
(734736, 19349)


In [18]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_vectors_tfidf, y_train)

RandomForestRegressor(random_state=42)

In [19]:
y_pred = rf.predict(X_test_vectors_tfidf)

In [20]:
y_pred

array([ 956.98581338, 6089.9911779 ,  948.62100681, ...,  515.50834634,
        699.40702201,  552.11086429])

In [21]:
len(y_pred)

734736

In [22]:
d = {"PRODUCT_LENGTH" : y_pred}

In [23]:
df1 = pd.DataFrame(data=d)
df1

,PRODUCT_LENGTH
0,956.985813
1,6089.991178
2,948.621007
3,298.575117
4,887.192752
...,...
734731,960.165504
734732,1196.967315
734733,515.508346
734734,699.407022


In [24]:
df_new = pd.concat([df_test, df1], axis = 1)
df_new.head()

,Title,PRODUCT_ID,PRODUCT_LENGTH
0,manuel d'héliogravure et de photogravure en re...,604373,956.985813
1,dcgaring microfiber throw blanket warm fuzzy p...,1729783,6089.991178
2,i-match auto parts front license plate bracket...,1871949,948.621007
3,pinmart gold plated excellence in service 1 ye...,1107571,298.575117
4,visual mathematics illustrated by the ti-92 an...,624253,887.192752


In [26]:
l = ["PRODUCT_ID", "PRODUCT_LENGTH"]
df_new = df_new[l]
df_new

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,956.985813
1,1729783,6089.991178
2,1871949,948.621007
3,1107571,298.575117
4,624253,887.192752
...,...,...
734731,921419,960.165504
734732,2456362,1196.967315
734733,841529,515.508346
734734,1190194,699.407022


In [28]:
df_new.to_csv("dataset/submission.csv", index = False, header = True)

In [31]:
path = "dataset/submission.csv"
df_sub = pd.read_csv(path)
df_sub.head()

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,956.985813
1,1729783,6089.991178
2,1871949,948.621007
3,1107571,298.575117
4,624253,887.192752


In [32]:
df_sub.isna().sum()

PRODUCT_ID        0
PRODUCT_LENGTH    0
dtype: int64

In [33]:
df_sub.describe()

,PRODUCT_ID,PRODUCT_LENGTH
count,7.347360e+05,7.347360e+05
mean,1.493725e+06,2.614789e+03
std,8.669773e+05,4.655967e+04
min,0.000000e+00,1.000000e+01
25%,7.396735e+05,6.601234e+02
50%,1.492776e+06,7.924726e+02
75%,2.242406e+06,1.090167e+03
max,2.999998e+06,3.208931e+06


In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

# evaluate the model
mse = mean_squared_error(X_test_vectors_tfidf, y_pred)
mae = mean_absolute_error(X_test_vectors_tfidf, y_pred)
r2 = r2_score(X_test_vectors_tfidf, y_pred)

# use cross-validation to evaluate the model
cv_scores = cross_val_score(rf, df_train[['Title']], df_train['PRODUCT_LENGTH'], cv=5)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)
print('Cross-validation scores:', cv_scores)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [35]:
y_pred = rf.predict(X_train_vectors_tfidf)

In [36]:
y_pred

array([ 707.79448752,  823.32448748,  935.04283765, ...,   87.4062144 ,
       1213.53661412, 9077.41203819])

In [38]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

# evaluate the model
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

# use cross-validation to evaluate the model
#cv_scores = cross_val_score(rf, df_train[['Title']], df_train['PRODUCT_LENGTH'], cv=5)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)
#print('Cross-validation scores:', cv_scores)

MSE: 1663053273.3306117
MAE: 1384.2004207497566
R-squared: 0.8096625874798853


In [39]:
# Calculate the R^2 score of the predictions
r2 = r2_score(y_train, y_pred)

# Print the R^2 score
print("Accuracy of the random forest regressor model: {:.2f}%".format(r2*100))

Accuracy of the random forest regressor model: 80.97%
